In [1]:
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt # plotting
import numpy as np # linear algebra
import os # accessing directory structure
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


In [2]:
for dirname, _, filenames in os.walk('/Users/shikarichacha/Desktop/air 1/Spanish stocks/spain/prices'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/Users/shikarichacha/Desktop/air 1/Spanish stocks/spain/prices/PQR.csv
/Users/shikarichacha/Desktop/air 1/Spanish stocks/spain/prices/BIOS.csv
/Users/shikarichacha/Desktop/air 1/Spanish stocks/spain/prices/TLGO.csv
/Users/shikarichacha/Desktop/air 1/Spanish stocks/spain/prices/NXTE.csv
/Users/shikarichacha/Desktop/air 1/Spanish stocks/spain/prices/YTEM.csv
/Users/shikarichacha/Desktop/air 1/Spanish stocks/spain/prices/YORS.csv
/Users/shikarichacha/Desktop/air 1/Spanish stocks/spain/prices/GRLS.csv
/Users/shikarichacha/Desktop/air 1/Spanish stocks/spain/prices/FAE.csv
/Users/shikarichacha/Desktop/air 1/Spanish stocks/spain/prices/MEDC.csv
/Users/shikarichacha/Desktop/air 1/Spanish stocks/spain/prices/CASHP.csv
/Users/shikarichacha/Desktop/air 1/Spanish stocks/spain/prices/YHAB.csv
/Users/shikarichacha/Desktop/air 1/Spanish stocks/spain/prices/BRKM3.csv
/Users/shikarichacha/Desktop/air 1/Spanish stocks/spain/prices/A3M.csv
/Users/shikarichacha/Desktop/air 1/Spanish stocks/spain/prices/VI

In [3]:
# Distribution graphs (histogram/bar graph) of column data
def plotPerColumnDistribution(df, nGraphShown, nGraphPerRow):
    nunique = df.nunique()
    df = df[[col for col in df if nunique[col] > 1 and nunique[col] < 50]] # For displaying purposes, pick columns that have between 1 and 50 unique values
    nRow, nCol = df.shape
    columnNames = list(df)
    nGraphRow = (nCol + nGraphPerRow - 1) / nGraphPerRow
    plt.figure(num = None, figsize = (6 * nGraphPerRow, 8 * nGraphRow), dpi = 80, facecolor = 'w', edgecolor = 'k')
    for i in range(min(nCol, nGraphShown)):
        plt.subplot(nGraphRow, nGraphPerRow, i + 1)
        columnDf = df.iloc[:, i]
        if (not np.issubdtype(type(columnDf.iloc[0]), np.number)):
            valueCounts = columnDf.value_counts()
            valueCounts.plot.bar()
        else:
            columnDf.hist()
        plt.ylabel('counts')
        plt.xticks(rotation = 90)
        plt.title(f'{columnNames[i]} (column {i})')
    plt.tight_layout(pad = 1.0, w_pad = 1.0, h_pad = 1.0)
    plt.show()


In [4]:
# Correlation matrix
def plotCorrelationMatrix(df, graphWidth):
    filename = df.dataframeName
    df = df.dropna('columns') # drop columns with NaN
    df = df[[col for col in df if df[col].nunique() > 1]] # keep columns where there are more than 1 unique values
    if df.shape[1] < 2:
        print(f'No correlation plots shown: The number of non-NaN or constant columns ({df.shape[1]}) is less than 2')
        return
    corr = df.corr()
    plt.figure(num=None, figsize=(graphWidth, graphWidth), dpi=80, facecolor='w', edgecolor='k')
    corrMat = plt.matshow(corr, fignum = 1)
    plt.xticks(range(len(corr.columns)), corr.columns, rotation=90)
    plt.yticks(range(len(corr.columns)), corr.columns)
    plt.gca().xaxis.tick_bottom()
    plt.colorbar(corrMat)
    plt.title(f'Correlation Matrix for {filename}', fontsize=15)
    plt.show()


In [5]:
# Scatter and density plots
def plotScatterMatrix(df, plotSize, textSize):
    df = df.select_dtypes(include =[np.number]) # keep only numerical columns
    # Remove rows and columns that would lead to df being singular
    df = df.dropna('columns')
    df = df[[col for col in df if df[col].nunique() > 1]] # keep columns where there are more than 1 unique values
    columnNames = list(df)
    if len(columnNames) > 10: # reduce the number of columns for matrix inversion of kernel density plots
        columnNames = columnNames[:10]
    df = df[columnNames]
    ax = pd.plotting.scatter_matrix(df, alpha=0.75, figsize=[plotSize, plotSize], diagonal='kde')
    corrs = df.corr().values
    for i, j in zip(*plt.np.triu_indices_from(ax, k = 1)):
        ax[i, j].annotate('Corr. coef = %.3f' % corrs[i, j], (0.8, 0.2), xycoords='axes fraction', ha='center', va='center', size=textSize)
    plt.suptitle('Scatter and Density Plot')
    plt.show()


Now you're ready to read in the data and use the plotting functions to visualize the data.

### Let's check 1st file: /kaggle/input/spain/stocks_list.csv

In [6]:
nRowsRead = 1000 # specify 'None' if want to read whole file
# stocks_list.csv may have more rows in reality, but we are only loading/previewing the first 1000 rows
df1 = pd.read_csv('/Users/shikarichacha/Desktop/air 1/Spanish stocks/spain/stocks_list.csv', delimiter=',', nrows = nRowsRead)
df1.dataframeName = 'stocks_list.csv'
nRow, nCol = df1.shape
print(f'There are {nRow} rows and {nCol} columns')

There are 234 rows and 7 columns


Let's take a quick look at what the data looks like:

In [7]:
df1.head(5)

,Unnamed: 0,country,name,full_name,isin,currency,symbol
0,0,spain,ACS,ACS Actividades de Construccion y Servicios SA,ES0167050915,EUR,ACS
1,1,spain,Abengoa,Abengoa,ES0105200416,EUR,ABG
2,2,spain,Atresmedia,Atresmedia,ES0109427734,EUR,A3M
3,3,spain,Acerinox,Acerinox,ES0132105018,EUR,ACX
4,4,spain,BBVA,BBVA,ES0113211835,EUR,BBVA


Distribution graphs (histogram/bar graph) of sampled columns:

In [8]:
plotPerColumnDistribution(df1, 10, 5)

<Figure size 2400x512 with 0 Axes>

### Let's check 2nd file: /kaggle/input/spain/prices/ACS.csv

In [9]:
nRowsRead = 1000 # specify 'None' if want to read whole file
# ACS.csv may have more rows in reality, but we are only loading/previewing the first 1000 rows
df2 = pd.read_csv('/kaggle/input/spain/prices/ACS.csv', delimiter=',', nrows = nRowsRead)
df2.dataframeName = 'ACS.csv'
nRow, nCol = df2.shape
print(f'There are {nRow} rows and {nCol} columns')

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/spain/prices/ACS.csv'

Let's take a quick look at what the data looks like:

In [ ]:
df2.head(5)

Distribution graphs (histogram/bar graph) of sampled columns:

In [ ]:
plotPerColumnDistribution(df2, 10, 5)

Correlation matrix:

In [ ]:
plotCorrelationMatrix(df2, 8)

Scatter and density plots:

In [ ]:
plotScatterMatrix(df2, 15, 10)

### Let's check 3rd file: /kaggle/input/spain/prices/ALFAA.csv

In [ ]:
nRowsRead = 1000 # specify 'None' if want to read whole file
# ALFAA.csv may have more rows in reality, but we are only loading/previewing the first 1000 rows
df3 = pd.read_csv('/kaggle/input/spain/prices/ALFAA.csv', delimiter=',', nrows = nRowsRead)
df3.dataframeName = 'ALFAA.csv'
nRow, nCol = df3.shape
print(f'There are {nRow} rows and {nCol} columns')

Let's take a quick look at what the data looks like:

In [ ]:
df3.head(5)

Distribution graphs (histogram/bar graph) of sampled columns:

In [ ]:
plotPerColumnDistribution(df3, 10, 5)

Correlation matrix:

In [ ]:
plotCorrelationMatrix(df3, 8)

Scatter and density plots:

In [ ]:
plotScatterMatrix(df3, 15, 10)

## Conclusion
This concludes your starter analysis! To go forward from here, click the blue "Fork Notebook" button at the top of this kernel. This will create a copy of the code and environment for you to edit. Delete, modify, and add code as you please. Happy Kaggling!